In [1]:
import torch
import numpy as np
import time
import sys
from torch.optim.lr_scheduler import StepLR
from os.path import abspath, dirname, exists
sys.path.append(abspath(dirname('demo')))
from module import MADE, TraDE_binary, NADE,  Errormodel, mod2, Loading_code, read_code, Abstractcode

In [2]:
def forward(n_s, m, van, syndrome, device, dtype, k=1):
    condition = syndrome*2-1
    x = (van.partial_forward(n_s=n_s, condition=condition, device=device, dtype=dtype, k=k) + 1)/2
    x = x[:, m:m+2*k]
    return x

In [16]:
'''Code para'''
c_type = 'rep'
n, d, k, seed = 31 , 31, 1, 0
info = read_code(d=d, k=k, n=n, seed=seed, c_type=c_type)
Code = Loading_code(info)


device = 'cuda:4'
dtype = torch.float32
m2 = mod2(device=device, dtype=dtype)

'''Errormodel'''
er = 0.001
E = Errormodel(er, e_model='dep')

'''Draw samples'''
trials = 100000
errors = E.generate_error(Code.n,  m=trials, seed=seed)
syndrome = m2.commute(errors, Code.g_stabilizer)
pe = E.pure(Code.pure_es, syndrome, device=device, dtype=dtype)
logical_error = m2.commute(errors, Code.logical_opt)
# print(logical_error)

# Training

In [14]:
'''net para'''
epoch, batch, lr = 100000, 10000, 0.001
ni = Code.m+2*k
depth, width = 4, 30

van = MADE(n=ni, depth=depth, width=width, residual=False).to(device).to(dtype)

optimizer = torch.optim.Adam(van.parameters(), lr=lr)#, momentum=0.9)
scheduler = StepLR(optimizer, step_size=2000, gamma=0.9)
# loss_his = []
# lo_his = []
for l in range(epoch):
    ers = E.generate_error(Code.n, m=batch, seed=False)

    configs = torch.hstack([m2.commute(ers, Code.g_stabilizer), m2.commute(ers, Code.logical_opt)]).to(device).to(dtype)

    # E.configs(sta=Code.g_stabilizer, log=Code.logical_opt, pe=pe, opts=ers).to(device).to(dtype)
    # print(configs[10, Code.m:ni])
    logp = van.log_prob((configs[:, :ni])*2-1) 
    loss = torch.mean((-logp), dim=0)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if optimizer.state_dict()['param_groups'][0]['lr'] > 0.0002 :
        scheduler.step()
    if (l+1)%1000==0:
        print('Loss:', loss)
    if (l+1) % 10000 == 0:
        lconf = forward(n_s=trials, m=Code.m, van=van, syndrome=syndrome, device=device, dtype=dtype, k=k)
        # print(lconf)

        '''correction'''
        # L = m2.confs_to_opt(confs=lconf, gs=Code.logical_opt)
        # recover = m2.opt_prod(pe, L)
        # # print(m2.commute(recover, Code.logical_opt))
        # check = m2.opt_prod(recover, errors)
        # commute = m2.commute(check, Code.logical_opt)
        # # print(commute)
        # fail = torch.count_nonzero(commute.sum(1))

        fail = torch.count_nonzero((lconf-logical_error).sum(1))

        logical_error_rate = fail/trials
        print(l, 'LER:', logical_error_rate)

Loss: tensor(0.5141, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4749, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4621, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4797, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4653, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4629, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4928, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4779, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4875, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.4409, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
9999 LER: tensor(0.0296, device='cuda:4')
Loss: tensor(0.4441, device='cuda:4', dtype=torch.float32, grad_fn=<MeanBackward1>)
Loss: tensor(0.5015, device='cuda:

# Decoding

In [17]:
from pymatching import Matching

def mwpm(Code, errors, er):
    pcm = Code.PCM.cpu().numpy()
    l1 = m2.rep(Code.logical_opt).int().cpu().numpy()
    l = np.zeros_like(l1)
    l[:, :n], l[:, n:] = l1[:, n:], l1[:, :n]

    weights = np.ones(pcm.shape[1])*np.log((1-er)/er)
    Decoder = Matching(pcm, weights=weights)
    syndrome = m2.commute(errors, Code.g_stabilizer)
    syndrome = syndrome.cpu().numpy()

    error = m2.rep(errors).squeeze().int().cpu().numpy()
    
    correct_number = 0
    t = 0
    for j in range(trials):
        e = error[j]
        s = syndrome[j]

        t1 = time.time()
        #print(s)
        recover = Decoder.decode(s)
        check = (e + recover)%2
        s = np.sum((check @ l.T) %2)
        t2 = time.time()
        t = t+(t2-t1)
        if s == 0:
            correct_number+=1
        
    lorate = 1 - correct_number/trials
    return lorate

lorate = mwpm(Code, errors, er)
print(lorate)

0.020469999999999988


In [ ]:
from module import exact_config

c_type = 'rep'
n, d, k, seed = 3 , 3, 1, 0
info = read_code(d=d, k=k, n=n, seed=seed, c_type=c_type)
Code = Loading_code(info)

device = 'cuda:4'
dtype = torch.float64
m2 = mod2(device=device, dtype=dtype)



configs = exact_config(D=2, N=2**2)
stas = m2.confs_to_opt(confs=configs, gs = Code.g_stabilizer)

trials = 100
E=Errormodel(e_rate=0.5, e_model='dep')
errors = E.generate_error(n, trials)
# print(errors)
non_zero = 0
for i in range(trials):
    log_pcoset = torch.sum(E.log_probability(m2.opt_prod(errors[i], stas), device=device, dtype=dtype))
    log_pcoset_z = torch.sum(E.log_probability(m2.opt_prod(m2.opt_prod(errors[i], Code.logical_opt[0]), stas), device=device, dtype=dtype))
    if log_pcoset-log_pcoset_z !=0:
        non_zero+=1
print(non_zero)





tensor([[1., 1., 1.],
        [2., 2., 2.]])
0
